In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from net_archs import LSTMModel
from sklearn.metrics import mean_squared_error,  roc_auc_score, f1_score
import datetime
import os
from sklearn.model_selection import train_test_split

In [2]:
print('main1_lstm_federated')

main1_lstm_federated


In [3]:
ts = torch.load('./dataset/ts.pt')
ts_labels = torch.load('./dataset/ts_labels.pt')

In [4]:
def ordinal_criterion(predictions, targets):
  # Ordinal regression with encoding as in https://arxiv.org/pdf/0704.1028.pdf

  # Create out modified target with [batch_size, num_labels] shape
  modified_target = torch.zeros_like(predictions)

  # Fill in ordinal target function, i.e. 0 -> [1,0,0,...]
  for i, target in enumerate(targets):
    modified_target[i, 0:int(target)+1] = 1

  return nn.MSELoss(reduction='none')(predictions, modified_target).sum(axis=1)

In [5]:
# Define train and test function
def train(net, optimizer, x, y, num_epoch=256, batch_size=16, criterion=ordinal_criterion):
  print_every = 10

  for n in range(num_epoch):
    # Mini batch sgd
    permutation = torch.randperm(x.size()[0])
    for i in range(0, x.size()[0], batch_size):
      indices = permutation[i:i+batch_size]
      x_mini, y_mini = x[indices], y[indices]
      y_pred = net(x_mini)
      loss = criterion(y_pred.float(), y_mini.float()) # default criterion = ordinal_criterion
      optimizer.zero_grad()
      loss.mean().backward()
      optimizer.step()

    if (n + 1) % print_every == 0:
      print(f'Epoch: {n + 1}, Loss: {loss.sum()}')

def prediction2label(pred: np.ndarray):
  """Convert ordinal predictions to class labels, e.g.
  [0.9, 0.1, 0.1, 0.1] -> 0
  [0.9, 0.9, 0.1, 0.1] -> 1
  [0.9, 0.9, 0.9, 0.1] -> 2
  etc.
  """
  return (pred > 0.5).cumprod(axis=1).sum(axis=1) - 1

def test(net, x_test, y_test):
  y_pred = net(x_test)
  if y_pred.shape[1] == 21:
    y_pred = prediction2label(net(x_test))
  else:
    y_pred = torch.clip(y_pred.round(), min=0, max=21)

  y_default_test = np.where(y_test.detach().numpy() > 9, 1, 0)
  y_default_pred = np.where(y_pred.detach().numpy() > 9, 1, 0)
  mse = mean_squared_error(y_test.detach().numpy(), y_pred.detach().numpy())
  mse_norm = mean_squared_error(y_test.detach().numpy() / 21, y_pred.detach().numpy() / 21)
  auc = roc_auc_score(y_default_test, y_default_pred)
  f1 = f1_score(y_default_test, y_default_pred)
  return mse, mse_norm, auc, f1


In [6]:
# Define LSTM FL simulation wrapper
from fl_simu import lstm_simulation

def run_lstm_simulation(split, X_train, y_train, X_test, y_test, dir='default', num_epoch=256, batch_size=16, num_rounds=15):
    return lstm_simulation(split, X_train, y_train, X_test, y_test, dir, num_epoch, batch_size, num_rounds)


D:\FD\venv\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [7]:
from sklearn.model_selection import KFold

num_epochs = 4
batch_size = 4

splitting = [0.5, 0.5]

def do_fl(splitting, X=ts, y=ts_labels, num_rounds=40):
    test_name = 'Dataset 1 LSTM'
    directory_path = 'lstm_model_checkpoints/' + test_name

    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    start = datetime.datetime.now()
    mses = []
    aucs = []

    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    print(type(X))
    print(type(y))
    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X, y)):
        print(f"Fold {fold_idx + 1}:")

        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
        y_test = torch.reshape(torch.tensor(y_test), (-1, 1))


        model_fl = run_lstm_simulation(splitting, X_train, y_train, X_test, y_test, test_name, num_epoch=num_epochs, batch_size=batch_size, num_rounds=num_rounds)

        mse, mse_norm, auc, f1 = test(model_fl, X_test, y_test)
        mses.append(mse)
        aucs.append(auc)

    end = datetime.datetime.now()
    time = end - start

    print('splitting:', splitting)
    print('mse:', mses)
    print('auc: ', aucs)
    print('Training time: ', time)

In [12]:
#20/10/5 clients rounds=40

In [13]:
split = [0.5, 0.5]
do_fl(split, num_rounds=20)

C:\Users\Shuyao\AppData\Local\Temp\ipykernel_11996\3107724849.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_11996\3107724849.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.reshape(torch.tensor(y_test), (-1, 1))
INFO flwr 2023-08-29 07:51:41,025 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=1, round_timeout=None)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([370, 3, 23])
Client 1 trainset shape torch.Size([350, 3, 23])


2023-08-29 07:51:47,732	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


KeyboardInterrupt: 

In [ ]:
split = [0.6, 0.4]
do_fl(split, num_rounds=20)

In [ ]:
split = [0.8, 0.2]
do_fl(split, num_rounds=20)

In [ ]:
split = [0.34, 0.33, 0.33]
do_fl(split, num_rounds=30)

In [ ]:
split = [0.6, 0.2, 0.2]
do_fl(split, num_rounds=30)

In [ ]:
split = [0.8, 0.1, 0.1]
do_fl(split, num_rounds=30)

In [ ]:
split = [0.2, 0.2, 0.2, 0.2, 0.2]
do_fl(split)

C:\Users\Shuyao\AppData\Local\Temp\ipykernel_6156\3421847257.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_6156\3421847257.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.reshape(torch.tensor(y_test), (-1, 1))
INFO flwr 2023-08-29 12:17:36,400 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=40, round_timeout=None)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([150, 3, 23])
Client 1 trainset shape torch.Size([150, 3, 23])
Client 2 trainset shape torch.Size([140, 3, 23])
Client 3 trainset shape torch.Size([140, 3, 23])
Client 4 trainset shape torch.Size([140, 3, 23])


2023-08-29 12:17:43,275	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 
INFO flwr 2023-08-29 12:17:49,594 | app.py:179 | Flower VCE: Ray initialized with resources: {'CPU': 8.0, 'node:127.0.0.1': 1.0, 'memory': 1568140494.0, 'object_store_memory': 784070246.0, 'GPU': 1.0}
INFO flwr 2023-08-29 12:17:49,595 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-29 12:17:49,596 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-29 12:17:52,571 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-29 12:17:52,572 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-29 12:17:52,573 | server.py:104 | FL starting
DEBUG flwr 2023-08-29 12:17:52,574 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-29 12:18:04,143 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-29 12:18:04,175 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...


DEBUG flwr 2023-08-29 12:18:04,532 | server.py:187 | evaluate_round 1 received 2 results and 0 failures
DEBUG flwr 2023-08-29 12:18:04,534 | server.py:222 | fit_round 2: strategy sampled 2 clients (out of 5)


...

DEBUG flwr 2023-08-29 12:36:12,476 | server.py:187 | evaluate_round 40 received 2 results and 0 failures
INFO flwr 2023-08-29 12:36:12,478 | server.py:153 | FL finished in 343.72874749999755
INFO flwr 2023-08-29 12:36:12,481 | app.py:225 | app_fit: losses_distributed [(1, 13.794444444444444), (2, 7.3), (3, 3.838888888888889), (4, 2.35), (5, 1.711111111111111), (6, 1.711111111111111), (7, 1.3944444444444444), (8, 1.1055555555555556), (9, 1.1222222222222222), (10, 1.0), (11, 1.0722222222222222), (12, 1.0333333333333334), (13, 0.85), (14, 0.9777777777777777), (15, 0.9111111111111111), (16, 0.8166666666666667), (17, 0.7611111111111111), (18, 0.7166666666666667), (19, 0.7444444444444445), (20, 0.75), (21, 0.7166666666666667), (22, 0.75), (23, 0.7555555555555555), (24, 0.7111111111111111), (25, 0.6722222222222223), (26, 0.7277777777777777), (27, 0.7222222222222222), (28, 1.0277777777777777), (29, 0.6388888888888888), (30, 0.7277777777777777), (31, 0.7388888888888889), (32, 0.8777777777777778

agrregate_evaluate results list length is  2
server round is  40
loss is  0.6166666666666667
History (loss, distributed):
	round 1: 13.794444444444444
	round 2: 7.3
	round 3: 3.838888888888889
	round 4: 2.35
	round 5: 1.711111111111111
	round 6: 1.711111111111111
	round 7: 1.3944444444444444
	round 8: 1.1055555555555556
	round 9: 1.1222222222222222
	round 10: 1.0
	round 11: 1.0722222222222222
	round 12: 1.0333333333333334
	round 13: 0.85
	round 14: 0.9777777777777777
	round 15: 0.9111111111111111
	round 16: 0.8166666666666667
	round 17: 0.7611111111111111
	round 18: 0.7166666666666667
	round 19: 0.7444444444444445
	round 20: 0.75
	round 21: 0.7166666666666667
	round 22: 0.75
	round 23: 0.7555555555555555
	round 24: 0.7111111111111111
	round 25: 0.6722222222222223
	round 26: 0.7277777777777777
	round 27: 0.7222222222222222
	round 28: 1.0277777777777777
	round 29: 0.6388888888888888
	round 30: 0.7277777777777777
	round 31: 0.7388888888888889
	round 32: 0.8777777777777778
	round 33: 0.688

C:\Users\Shuyao\AppData\Local\Temp\ipykernel_6156\3421847257.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.reshape(torch.tensor(y_test), (-1, 1))
INFO flwr 2023-08-29 12:36:12,800 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=40, round_timeout=None)


Client 0 trainset shape torch.Size([150, 3, 23])
Client 1 trainset shape torch.Size([150, 3, 23])
Client 2 trainset shape torch.Size([141, 3, 23])
Client 3 trainset shape torch.Size([140, 3, 23])
Client 4 trainset shape torch.Size([140, 3, 23])


2023-08-29 12:36:21,593	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


In [ ]:
split = [0.6, 0.1, 0.1, 0.1, 0.1]
do_fl(split)

In [ ]:
split = [0.8, 0.05, 0.05, 0.05, 0.05]
do_fl(split)

In [15]:
split = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
do_fl(split)

C:\Users\Shuyao\AppData\Local\Temp\ipykernel_11996\3421847257.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_11996\3421847257.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.reshape(torch.tensor(y_test), (-1, 1))
INFO flwr 2023-08-29 08:00:08,429 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=40, round_timeout=None)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([75, 3, 23])
Client 1 trainset shape torch.Size([75, 3, 23])
Client 2 trainset shape torch.Size([75, 3, 23])
Client 3 trainset shape torch.Size([75, 3, 23])
Client 4 trainset shape torch.Size([70, 3, 23])
Client 5 trainset shape torch.Size([70, 3, 23])
Client 6 trainset shape torch.Size([70, 3, 23])
Client 7 trainset shape torch.Size([70, 3, 23])
Client 8 trainset shape torch.Size([70, 3, 23])
Client 9 trainset shape torch.Size([70, 3, 23])


2023-08-29 08:00:16,969	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-08-29 08:00:21,345 | app.py:179 | Flower VCE: Ray initialized with resources: {'CPU': 8.0, 'object_store_memory': 1672074854.0, 'node:127.0.0.1': 1.0, 'memory': 3344149710.0, 'GPU': 1.0}
INFO flwr 2023-08-29 08:00:21,347 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-29 08:00:21,348 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-29 08:00:24,945 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-29 08:00:24,947 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-29 08:00:24,948 | server.py:104 | FL starting
DEBUG flwr 2023-08-29 08:00:24,948 | server.py:222 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-29 08:00:35,559 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-08-29 08:00:35,682 | fedav

agrregate_fit results list length is  5
Saving round 1 aggregated_parameters...


DEBUG flwr 2023-08-29 08:00:36,245 | server.py:187 | evaluate_round 1 received 5 results and 0 failures
DEBUG flwr 2023-08-29 08:00:36,246 | server.py:222 | fit_round 2: strategy sampled 5 clients (out of 10)


...

agrregate_evaluate results list length is  5
server round is  39
loss is  0.6777777777777778


DEBUG flwr 2023-08-29 08:26:17,942 | server.py:236 | fit_round 40 received 5 results and 0 failures
DEBUG flwr 2023-08-29 08:26:18,080 | server.py:173 | evaluate_round 40: strategy sampled 5 clients (out of 10)


agrregate_fit results list length is  5
Saving round 40 aggregated_parameters...


DEBUG flwr 2023-08-29 08:26:18,630 | server.py:187 | evaluate_round 40 received 5 results and 0 failures
INFO flwr 2023-08-29 08:26:18,632 | server.py:153 | FL finished in 307.14734119999775
INFO flwr 2023-08-29 08:26:18,636 | app.py:225 | app_fit: losses_distributed [(1, 10.105555555555556), (2, 8.544444444444444), (3, 6.8), (4, 4.427777777777778), (5, 3.1444444444444444), (6, 1.788888888888889), (7, 1.6111111111111112), (8, 1.2833333333333337), (9, 1.05), (10, 1.0277777777777777), (11, 1.0222222222222221), (12, 0.8166666666666667), (13, 0.8), (14, 0.8388888888888889), (15, 0.8777777777777778), (16, 0.8444444444444444), (17, 0.85), (18, 0.8166666666666667), (19, 0.85), (20, 0.8555555555555555), (21, 0.7777777777777778), (22, 0.7888888888888889), (23, 0.8055555555555556), (24, 0.7555555555555555), (25, 0.75), (26, 0.8055555555555556), (27, 0.7777777777777778), (28, 0.6888888888888889), (29, 0.6999999999999998), (30, 0.6999999999999998), (31, 0.6777777777777778), (32, 0.6666666666666666

agrregate_evaluate results list length is  5
server round is  40
loss is  0.6444444444444446
History (loss, distributed):
	round 1: 10.105555555555556
	round 2: 8.544444444444444
	round 3: 6.8
	round 4: 4.427777777777778
	round 5: 3.1444444444444444
	round 6: 1.788888888888889
	round 7: 1.6111111111111112
	round 8: 1.2833333333333337
	round 9: 1.05
	round 10: 1.0277777777777777
	round 11: 1.0222222222222221
	round 12: 0.8166666666666667
	round 13: 0.8
	round 14: 0.8388888888888889
	round 15: 0.8777777777777778
	round 16: 0.8444444444444444
	round 17: 0.85
	round 18: 0.8166666666666667
	round 19: 0.85
	round 20: 0.8555555555555555
	round 21: 0.7777777777777778
	round 22: 0.7888888888888889
	round 23: 0.8055555555555556
	round 24: 0.7555555555555555
	round 25: 0.75
	round 26: 0.8055555555555556
	round 27: 0.7777777777777778
	round 28: 0.6888888888888889
	round 29: 0.6999999999999998
	round 30: 0.6999999999999998
	round 31: 0.6777777777777778
	round 32: 0.6666666666666666
	round 33: 0.716

In [16]:
split = [0.6, 0.05, 0.05, 0.05, 0.05, 0.04, 0.04, 0.04, 0.04, 0.04]
do_fl(split)

C:\Users\Shuyao\AppData\Local\Temp\ipykernel_11996\3421847257.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_11996\3421847257.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.reshape(torch.tensor(y_test), (-1, 1))
INFO flwr 2023-08-29 08:26:18,932 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=40, round_timeout=None)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([440, 3, 23])
Client 1 trainset shape torch.Size([28, 3, 23])
Client 2 trainset shape torch.Size([28, 3, 23])
Client 3 trainset shape torch.Size([28, 3, 23])
Client 4 trainset shape torch.Size([28, 3, 23])
Client 5 trainset shape torch.Size([28, 3, 23])
Client 6 trainset shape torch.Size([28, 3, 23])
Client 7 trainset shape torch.Size([28, 3, 23])
Client 8 trainset shape torch.Size([28, 3, 23])
Client 9 trainset shape torch.Size([28, 3, 23])


2023-08-29 08:26:27,400	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-08-29 08:26:31,925 | app.py:179 | Flower VCE: Ray initialized with resources: {'CPU': 8.0, 'object_store_memory': 1043000524.0, 'node:127.0.0.1': 1.0, 'memory': 2086001051.0, 'GPU': 1.0}
INFO flwr 2023-08-29 08:26:31,926 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-29 08:26:31,927 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-29 08:26:35,061 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-29 08:26:35,062 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-29 08:26:35,062 | server.py:104 | FL starting
DEBUG flwr 2023-08-29 08:26:35,063 | server.py:222 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-29 08:26:41,943 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-08-29 08:26:42,062 | fedav

agrregate_fit results list length is  5
Saving round 1 aggregated_parameters...


DEBUG flwr 2023-08-29 08:26:42,578 | server.py:187 | evaluate_round 1 received 5 results and 0 failures
DEBUG flwr 2023-08-29 08:26:42,580 | server.py:222 | fit_round 2: strategy sampled 5 clients (out of 10)


...

agrregate_evaluate results list length is  5
server round is  39
loss is  0.5277777777777778


DEBUG flwr 2023-08-29 09:00:52,282 | server.py:236 | fit_round 40 received 5 results and 0 failures
DEBUG flwr 2023-08-29 09:00:52,411 | server.py:173 | evaluate_round 40: strategy sampled 5 clients (out of 10)


agrregate_fit results list length is  5
Saving round 40 aggregated_parameters...


DEBUG flwr 2023-08-29 09:00:52,938 | server.py:187 | evaluate_round 40 received 5 results and 0 failures
INFO flwr 2023-08-29 09:00:52,940 | server.py:153 | FL finished in 440.97048070000164
INFO flwr 2023-08-29 09:00:52,944 | app.py:225 | app_fit: losses_distributed [(1, 3.7444444444444445), (2, 1.25), (3, 0.8055555555555556), (4, 1.0055555555555555), (5, 0.8333333333333334), (6, 0.7611111111111111), (7, 0.7111111111111111), (8, 0.7055555555555557), (9, 0.65), (10, 0.6333333333333333), (11, 0.6388888888888887), (12, 0.7277777777777777), (13, 0.65), (14, 0.6999999999999998), (15, 0.6666666666666666), (16, 0.7833333333333333), (17, 0.6555555555555556), (18, 0.6944444444444444), (19, 0.65), (20, 0.6388888888888887), (21, 0.6166666666666667), (22, 0.5722222222222222), (23, 0.6388888888888887), (24, 0.6333333333333333), (25, 0.6055555555555554), (26, 0.5888888888888889), (27, 0.6999999999999998), (28, 0.5666666666666667), (29, 0.7666666666666667), (30, 0.5833333333333334), (31, 0.561111111

agrregate_evaluate results list length is  5
server round is  40
loss is  0.5055555555555555
History (loss, distributed):
	round 1: 3.7444444444444445
	round 2: 1.25
	round 3: 0.8055555555555556
	round 4: 1.0055555555555555
	round 5: 0.8333333333333334
	round 6: 0.7611111111111111
	round 7: 0.7111111111111111
	round 8: 0.7055555555555557
	round 9: 0.65
	round 10: 0.6333333333333333
	round 11: 0.6388888888888887
	round 12: 0.7277777777777777
	round 13: 0.65
	round 14: 0.6999999999999998
	round 15: 0.6666666666666666
	round 16: 0.7833333333333333
	round 17: 0.6555555555555556
	round 18: 0.6944444444444444
	round 19: 0.65
	round 20: 0.6388888888888887
	round 21: 0.6166666666666667
	round 22: 0.5722222222222222
	round 23: 0.6388888888888887
	round 24: 0.6333333333333333
	round 25: 0.6055555555555554
	round 26: 0.5888888888888889
	round 27: 0.6999999999999998
	round 28: 0.5666666666666667
	round 29: 0.7666666666666667
	round 30: 0.5833333333333334
	round 31: 0.5611111111111111
	round 32: 0.

In [17]:
split = [0.8, 0.03, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]
do_fl(split)

C:\Users\Shuyao\AppData\Local\Temp\ipykernel_11996\3421847257.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_11996\3421847257.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.reshape(torch.tensor(y_test), (-1, 1))
INFO flwr 2023-08-29 09:00:53,219 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=40, round_timeout=None)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([580, 3, 23])
Client 1 trainset shape torch.Size([14, 3, 23])
Client 2 trainset shape torch.Size([14, 3, 23])
Client 3 trainset shape torch.Size([14, 3, 23])
Client 4 trainset shape torch.Size([14, 3, 23])
Client 5 trainset shape torch.Size([14, 3, 23])
Client 6 trainset shape torch.Size([14, 3, 23])
Client 7 trainset shape torch.Size([14, 3, 23])
Client 8 trainset shape torch.Size([14, 3, 23])
Client 9 trainset shape torch.Size([14, 3, 23])


2023-08-29 09:01:01,309	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-08-29 09:01:05,895 | app.py:179 | Flower VCE: Ray initialized with resources: {'CPU': 8.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 1509739315.0, 'memory': 3019478631.0, 'GPU': 1.0}
INFO flwr 2023-08-29 09:01:05,896 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-29 09:01:05,897 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-29 09:01:08,873 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-29 09:01:08,874 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-29 09:01:08,874 | server.py:104 | FL starting
DEBUG flwr 2023-08-29 09:01:08,875 | server.py:222 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-29 09:01:30,884 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-08-29 09:01:30,961 | fedav

agrregate_fit results list length is  5
Saving round 1 aggregated_parameters...


DEBUG flwr 2023-08-29 09:01:31,367 | server.py:187 | evaluate_round 1 received 5 results and 0 failures
DEBUG flwr 2023-08-29 09:01:31,368 | server.py:222 | fit_round 2: strategy sampled 5 clients (out of 10)


...

agrregate_evaluate results list length is  5
server round is  39
loss is  0.5833333333333334


DEBUG flwr 2023-08-29 09:37:20,840 | server.py:236 | fit_round 40 received 5 results and 0 failures
DEBUG flwr 2023-08-29 09:37:20,977 | server.py:173 | evaluate_round 40: strategy sampled 5 clients (out of 10)


agrregate_fit results list length is  5
Saving round 40 aggregated_parameters...


DEBUG flwr 2023-08-29 09:37:21,453 | server.py:187 | evaluate_round 40 received 5 results and 0 failures
INFO flwr 2023-08-29 09:37:21,454 | server.py:153 | FL finished in 372.4624048000005
INFO flwr 2023-08-29 09:37:21,458 | app.py:225 | app_fit: losses_distributed [(1, 101.45), (2, 8.716666666666667), (3, 1.8277777777777777), (4, 0.8833333333333333), (5, 0.8277777777777777), (6, 0.7777777777777778), (7, 0.7944444444444444), (8, 0.7388888888888889), (9, 1.1444444444444444), (10, 0.7777777777777778), (11, 0.6388888888888887), (12, 0.6833333333333333), (13, 0.6555555555555556), (14, 0.6444444444444446), (15, 0.5944444444444444), (16, 0.6722222222222224), (17, 0.6333333333333333), (18, 0.6277777777777778), (19, 0.5666666666666667), (20, 0.8888888888888888), (21, 0.6999999999999998), (22, 0.5666666666666667), (23, 0.48333333333333334), (24, 0.5277777777777778), (25, 0.5444444444444444), (26, 0.5555555555555556), (27, 0.5555555555555556), (28, 0.55), (29, 0.5777777777777776), (30, 0.566666

agrregate_evaluate results list length is  5
server round is  40
loss is  0.5722222222222222
History (loss, distributed):
	round 1: 101.45
	round 2: 8.716666666666667
	round 3: 1.8277777777777777
	round 4: 0.8833333333333333
	round 5: 0.8277777777777777
	round 6: 0.7777777777777778
	round 7: 0.7944444444444444
	round 8: 0.7388888888888889
	round 9: 1.1444444444444444
	round 10: 0.7777777777777778
	round 11: 0.6388888888888887
	round 12: 0.6833333333333333
	round 13: 0.6555555555555556
	round 14: 0.6444444444444446
	round 15: 0.5944444444444444
	round 16: 0.6722222222222224
	round 17: 0.6333333333333333
	round 18: 0.6277777777777778
	round 19: 0.5666666666666667
	round 20: 0.8888888888888888
	round 21: 0.6999999999999998
	round 22: 0.5666666666666667
	round 23: 0.48333333333333334
	round 24: 0.5277777777777778
	round 25: 0.5444444444444444
	round 26: 0.5555555555555556
	round 27: 0.5555555555555556
	round 28: 0.55
	round 29: 0.5777777777777776
	round 30: 0.5666666666666667
	round 31: 0.